In [1]:
# Importing the necessary libraries

import imutils.contours
import argparse
import numpy as np
import glob
import cv2
import os

**Input collection**

In [2]:
# Checking if we are in the correct directory (the one from which we should read the test image)

os.getcwd()

'C:\\Users\\Aida\\Downloads'

In [3]:
# Reading the test image

image = cv2.imread(r'C:\\Users\\Aida\\Downloads\\flower.jpeg')
# Run the below code to see the image
# cv2.imshow("Image", image)
# cv2.waitKey(0)

In [4]:
# Giving the fixed object's width in mm as an input

width = 30

**Image Processing**

In [5]:
# Making the image gray for easier calculations, then making it more smooth

greyscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
greyscale = cv2.GaussianBlur(greyscale, (7, 7), 0)

In [6]:
# Finding the edges of the image

canny_output = cv2.Canny(greyscale, 50, 100)
canny_output = cv2.dilate(canny_output, None, iterations=1)
canny_output = cv2.erode(canny_output, None, iterations=1)

In [7]:
# Defining a function for selecting good parameters to have better edged image

def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = (lower, upper)
    # return the possible good parameters
    return edged

#auto_canny(image)

In [8]:
# Finding the contours of the objects, sorting them from left-to-right

contours, hierarchy = cv2.findContours(canny_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
(contours, hierarchy) = imutils.contours.sort_contours(contours)

# if you want to see the contours run the below code

# cv2.drawContours(image, contours, contourIdx = -1, color = (0, 255, 0), thickness = 3)
# cv2.imshow('Contours', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [9]:
# Finding the coordinates to draw bounding rectangles based on previosly founded contours

contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)

for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    
output_image = image.copy()
mmPerPixel = width / boundRect[0][2]
highestRect = 1
lowestRect = 0

for i in range(1, len(contours)):

    # if the rectangle is too small, continue
    if boundRect[i][2] < 50 or boundRect[i][3] < 50:
        continue

    # finding the highest and lowest rectangles
    if lowestRect > boundRect[i][1]:
        lowestRect = boundRect[i][1]
    if highestRect < (boundRect[i][1] + boundRect[i][3]):
        highestRect = (boundRect[i][1] + boundRect[i][3])

    # creating the boundary boxes
    cv2.rectangle(output_image, (int(boundRect[i][0]), int(boundRect[i][1])),
                  (int(boundRect[i][0] + boundRect[i][2]),
                  int(boundRect[i][1] + boundRect[i][3])), (255, 0, 0), 2)

**Measuring the height of the plant**

In [10]:
# Calculating the size of our plant

plantHeight = (highestRect - lowestRect) * mmPerPixel
print("Plant height is {0:.0f}mm".format(plantHeight))

Plant height is 190mm


In [11]:
# Resizing and displaying the image 
resized_image = cv2.resize(output_image, (1280, 720))
cv2.imshow("Image", resized_image)
cv2.waitKey(0)

107